In [8]:
import collections
import random
seed = 1
random.seed(seed)

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotly.express as px
import seaborn as sn
import xgboost as xgb

from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
#from boruta import BorutaPy
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve, roc_auc_score, accuracy_score
from hyperopt_opt import HPOpt


pd.set_option('display.max_columns', None)

# Define LightGBM callbacks
#callbacks = [xgb.early_stopping(10, verbose=0), lgb.log_evaluation(period=0)]

AttributeError: module 'xgboost' has no attribute 'early_stopping'

In [9]:
path = "MM_PredictivoFraude2.csv"

In [10]:
df = pd.read_csv(path, dtype=str, header=[0, 1], index_col=0)
df.head(1)

casos_cc                                                             \
       nsin       ncaso pos        nexp          nepi tipo         ccc   
0  29899501  1300298995   1  1300298995  2019E0001248  ANL  0810971910   

                                                                              \
  regimen           regccc situacion es_atipico es_recaida     hay_at_previo   
0    0521  0521/0810971910   Cerrado      False      False  No hay AT previo   

                                                                         \
  duracion dur_previo dur_std v_std     f_parte      f_baja  f_alta_adm   
0        6        0.0    15.0   9.0  2019-01-04  2019-01-02  2019-01-07   

                                  \
   f_alta_med med_clave cie9_cod   
0  2019-01-07       469    959.8   

                                                               \
                                          cie9_descr dele_cod   
0  TRAUMISMO NO ESPECIF. OTROS SITIOS ESPECIFICAD...      108   

                                                                  \
               dele_nombre d_sem_baja mes_baja es_trafico gestor   
0  DELEGACION DE BARCELONA  MIERCOLES    ENERO      False   1042   

                                                                  \
  es_pl_empleado es_pl_activo es_pago_dir es_susp_ext f_susp_ext   
0          False        False       False       False        NaN   

                                                                           \
  es_derecho_cobro es_alta_prev_insp es_recaida_ip_denegada es_covid  mpq   
0             True             False                  False    False  NaN   

   casos_ep                                                                    \
  es_atrium f_atrium prop_atrium f_resp_atrium resp_atrium f_ini_pd motivo_pd   
0     False      NaN         NaN           NaN         NaN      NaN       NaN   

                   aut                                                     \
  nrehab nbio registro ent_at sit_trab UG_TGSS sexo    FECHA_AT OPCION_IT   
0    0.0  0.0       01    007       52  082020    M  2017-07-01      True   

                                                                           \
  FECHA_OPCION MUTUA_IT FECHA_MUTUA_IT base_coti colectivo HORA_PRE N_RED   
0   2017-07-01      007     2017-07-01   2000.69   0200069      NaN   NaN   

                                                                      \
  i_auto nacion sist_es i_trade f_trade cnae9  cese      f_cese dele   
0  False    724   False   False     NaN  4726  True  2017-07-01  108   

               aut_fijo                                 criterios         \
  tipo_situ      f_nacm                       historico        c1     c2   
0       NaN  1979-02-12  [['01.07.2017', '31.01.2019']]     False  False   

                                                                         \
      c3     c4     c5     c6     c7     c8     c9    c10    c11    c12   
0  False  False  False  False  False  False  False  False  False  False   

                    detectives     creadas                          \
     c13    c14      resultado      f_alta      f_baja alta_oficio   
0  False  False  Sin detective  2017-07-01  2019-01-31           0   

                                                                          \
  prev_rechazado provincia   edad gr_edad naltas dist_ult_baja avg_altas   
0              0        08  39.92   36-45      1         10000     579.0   

                                criterios    creadas                        \
  nrecaidas ncasos n_dist_casos      peso d_sem_baja finde_baja es_nac_esp   
0         0      1            1         0          3      False       True   

                                        socioeconomicas                        \
  mes_baja next_fest past_fest nfest_14      empresas.0 empresas.1 empresas.2   
0        1       107        -1        3          470067      12281     166934   

                                                                   

In [11]:
socec_cols = []
for i in list(df.columns):
    if i[0] == 'socioeconomicas':
        socec_cols.append(i)

# df = df.drop(columns=socec_cols)

float_cols_ = [
     ('casos_cc', 'duracion'), ('casos_cc', 'dur_previo'), ('casos_cc', 'dur_std'), ('casos_cc', 'v_std'),
     ('casos_ep', 'nrehab'), ('casos_ep', 'nbio'), 
     ('aut', 'base_coti'),  
     ('creadas', 'alta_oficio'), ('creadas', 'prev_rechazado'), ('creadas', 'edad'), ('creadas', 'naltas'), ('creadas', 'dist_ult_baja'),
     ('creadas', 'avg_altas'), ('creadas', 'nrecaidas'), ('creadas', 'ncasos'), ('creadas', 'n_dist_casos'), ('criterios', 'peso'), ('creadas', 'next_fest'),
     ('creadas', 'past_fest'), ('creadas', 'nfest_14'), ('creadas', 'diff_baja_parte'), ('creadas', 'diff_ant_caso'), 
     ('creadas', 'diff_ant_med_adm'), ('creadas', 'mean_ant_med_adm'), ('creadas', 'max_ant_med_adm'), ('creadas', 'diff_baja_ini_pd'), 
     ('creadas', 'diff_baja_at'), ('creadas', 'diff_alta_at'), ('creadas', 'diff_baja_mutua'), ('creadas', 'diff_alta_mutua'), 
     ('creadas', 'diff_baja_trade'), ('creadas', 'diff_alta_trade'), ('creadas', 'diff_baja_cese'), ('creadas', 'diff_alta_cese'),
     ('creadas', 'dias_en_alta')
]

float_cols = float_cols_ + socec_cols

for col in float_cols:
     print(col)
     df[col] = df[col].astype({col: float})

('casos_cc', 'duracion')
('casos_cc', 'dur_previo')
('casos_cc', 'dur_std')
('casos_cc', 'v_std')
('casos_ep', 'nrehab')
('casos_ep', 'nbio')
('aut', 'base_coti')
('creadas', 'alta_oficio')
('creadas', 'prev_rechazado')
('creadas', 'edad')
('creadas', 'naltas')
('creadas', 'dist_ult_baja')
('creadas', 'avg_altas')
('creadas', 'nrecaidas')
('creadas', 'ncasos')
('creadas', 'n_dist_casos')
('criterios', 'peso')
('creadas', 'next_fest')
('creadas', 'past_fest')
('creadas', 'nfest_14')
('creadas', 'diff_baja_parte')
('creadas', 'diff_ant_caso')
('creadas', 'diff_ant_med_adm')
('creadas', 'mean_ant_med_adm')
('creadas', 'max_ant_med_adm')
('creadas', 'diff_baja_ini_pd')
('creadas', 'diff_baja_at')
('creadas', 'diff_alta_at')
('creadas', 'diff_baja_mutua')
('creadas', 'diff_alta_mutua')
('creadas', 'diff_baja_trade')
('creadas', 'diff_alta_trade')
('creadas', 'diff_baja_cese')
('creadas', 'diff_alta_cese')
('creadas', 'dias_en_alta')
('socioeconomicas', 'empresas.0')
('socioeconomicas', 'emp

('socioeconomicas', 'rvivienda_h.83')
('socioeconomicas', 'pvivienda_h.84')
('socioeconomicas', 'pvivienda_h.85')
('socioeconomicas', 'pvivienda_h.86')
('socioeconomicas', 'pvivienda_h.87')
('socioeconomicas', 'pvivienda_h.88')
('socioeconomicas', 'pobreza_exclusion.89')
('socioeconomicas', 'pobreza_exclusion.90')
('socioeconomicas', 'pobreza_exclusion.91')
('socioeconomicas', 'pobreza_exclusion.92')


----

Segmentamos la variable de `cie9`

In [ ]:
cie9 = list(df["cie9_cod"])

for i,cod in enumerate(list(cie9)):
    try:
        if type(cod) == float:
            continue
        elif "0000" in cod:
            continue
        else:
            cie9[i] = float(cod)
    except ValueError:
        #print(type(cod),"-->",cod)
        continue

strs=[]
for i in cie9:
    if type(i) == str:
        strs.append(i)

len(strs)

cie_cate = []
for cod in cie9:
    if type(cod) == str:
        if "V" in cod:
            cie_cate.append("V")
        elif "E" in cod:
            cie_cate.append("E")
        elif "0000" in cod:
            cie_cate.append("0000")
        else:
            cie_cate.append(cod)
    else:
        if cod < 140:
            cie_cate.append("001-139")
        elif cod < 240:
            cie_cate.append("140-239")
        elif cod < 280:
            cie_cate.append("240-279")
        elif cod < 290:
            cie_cate.append("280-289")
        elif cod < 320:
            cie_cate.append("290-319")
        elif cod < 390:
            cie_cate.append("320-389")
        elif cod < 460:
            cie_cate.append("390-459")
        elif cod < 520:
            cie_cate.append("460-519")
        elif cod < 580:
            cie_cate.append("520-579")
        elif cod < 630:
            cie_cate.append("580-629")
        elif cod < 680:
            cie_cate.append("630-679")
        elif cod < 710:
            cie_cate.append("680-709")
        elif cod < 740:
            cie_cate.append("710-739")
        elif cod < 760:
            cie_cate.append("740-759")
        elif cod < 780:
            cie_cate.append("760-779")
        elif cod < 800:
            cie_cate.append("780-779")
        elif cod < 1000:
            cie_cate.append("800-999")
        else:
            cie_cate.append("Other")

Pasamos a formato fecha la variables de alta médica y alta administrativa para crear la variable "prohibida".

In [ ]:
to_date = [
    ('casos_cc', 'f_alta_adm'), ('casos_cc', 'f_alta_med'), 
]

for col in to_date:
    print(col)
    df[col] = pd.to_datetime(df[col], format='%Y-%m-%d %H:%M:%S').dt.date

Rellenamos las fechas que faltan de `f_alta_med` con las de la variable `f_alta_adm`

In [ ]:
df[('casos_cc', 'f_alta_med')] = df[('casos_cc', 'f_alta_med')].fillna(df[('casos_cc', 'f_alta_adm')])

In [ ]:
df[('creadas','diff_alta_adm_med')] = (df[('casos_cc','f_alta_adm')] - df[('casos_cc','f_alta_med')]).dt.days
df[('creadas','diff_alta_adm_med')] = df[('creadas','diff_alta_adm_med')].astype({('creadas','diff_alta_adm_med'): float})

df[('creadas','diff_alta_adm_med_')] = [0 if i < 0 else i for i in df[('creadas','diff_alta_adm_med')]]
df[('creadas','diff_alta_adm_med_')] = df[('creadas','diff_alta_adm_med_')].astype({('creadas','diff_alta_adm_med_'): float})

df[('creadas','es_alta_adm_post_med')] = [1 if i > 0 else 0 for i in df[('creadas','diff_alta_adm_med')]]

df[('creadas','es_alta_adm_post_med2')] = [1 if i<0 else 0 for i in df[('creadas','diff_alta_adm_med')]]


float_cols = float_cols + [('creadas','diff_alta_adm_med'), ('creadas','diff_alta_adm_med_')]

Podemos eliminar ahora ya todas las fechas ya que no las necesitaremos más.

In [ ]:
to_drop = [
        ('casos_cc', 'f_baja'), ('casos_cc','f_parte'), ('casos_cc', 'f_alta_adm'), ('casos_cc', 'f_alta_med'), ('casos_cc','f_susp_ext'), 
        ('casos_ep','f_ini_pd'), 
        ('aut','FECHA_AT'), ('aut','FECHA_OPCION'), ('aut','FECHA_AT'), ('aut','FECHA_MUTUA_IT'), 
        ('aut','f_cese'), ('aut','f_trade'), 
        ('aut_fijo','f_nacm'),
        ('creadas','f_alta'), ('creadas','f_baja')
]

df = df.drop(columns=to_drop)

Eliminamos algunas de las variables que podemos eliminar si o si (o bien porque todos los valores son únicos o porque son todos iguales)

In [ ]:
to_drop = [
    ('casos_cc', 'pos'), ('casos_cc','ccc'), ('casos_cc','ncaso'), ('casos_cc','nexp'), ('casos_cc', 'nsin'), ('casos_cc', 'nepi'), ('casos_cc', 'regimen'), ('casos_cc', 'regccc'), ('casos_cc', 'situacion'), ('casos_cc', 'cie9_descr'), ('casos_cc', 'dele_nombre'), ('casos_cc', 'es_covid'), 
    ('aut', 'registro'), ('aut', 'sit_trab'), ('aut', 'OPCION_IT'), ('aut', 'MUTUA_IT'), ('aut', 'HORA_PRE'), ('aut', 'tipo_situ'), ('aut_fijo', 'historico'),
    ('criterios', 'c1'), ('criterios', 'c3'), ('criterios', 'c6'), ('criterios', 'c7'), ('criterios', 'c8'), ('criterios', 'c9'), ('criterios', 'c11'), ('criterios', 'c13'), ('criterios', 'c14'),
    ('casos_cc', 'd_sem_baja'), ('casos_cc', 'mes_baja'),
]

df = df.drop(columns=to_drop)

Eliminamos también las variables de ATRIUM porque estas las tenemos cuando pasa mucho tiempo de la baja laboral, queremos ser más rápidos en determinar el fraude.

In [ ]:
to_drop = [
    ('casos_ep', 'es_atrium'), ('casos_ep', 'f_atrium'), ('casos_ep', 'prop_atrium'), ('casos_ep', 'f_resp_atrium'), ('casos_ep', 'resp_atrium'),
    ('creadas', 'diff_alta_atrium'), ('creadas', 'prop_atrium_pos')
]

df = df.drop(columns=to_drop)

Eliminamos otras variables que no nos interesan:
* Variables con muchos NaN: `mpq`, `motivo_pd`
* Variables categoricas con muchas dispersión*: `UG_TGSS`, `colectivo`, `N_RED`, `med_clave`, `dele_cod`, `gestor`, `cnae9`, `dele`, `nacion`, `provincia`
* Variables que no tienen ningún registro en las variables con etiquetado negativo o bien en tampoco en las de positivo: `diff_baja_trade`, `diff_alta_trade`, `i_trade`, `c4`, `es_susp_ext`, `es_alta_prev_insp`, `es_atipico`, `es_trafico`, `es_recaida_ip_denegada`, `es_pl_empleado`, `baja_mutua_7`
* Variables relacionadas con la duración: `duración`, `dur_std`, `v_std`


(*) Además el problema de estas variables es que si sale una nueva categoría el modelo no va a ser capaz de entenderlo


In [ ]:
to_drop = [
    ('casos_cc', 'mpq'), ('casos_ep', 'nbio'), ('aut', 'UG_TGSS'), 
    ('aut', 'colectivo'), ('aut', 'N_RED'),

    ('casos_cc', 'med_clave'), ('casos_cc', 'dele_cod'), ('casos_cc', 'gestor'), 
    ('casos_ep', 'motivo_pd'),
    ('aut', 'cnae9'), ('aut', 'dele'), ('aut', 'nacion'),
    ('creadas', 'provincia'),


    ('creadas', 'diff_baja_trade'), ('creadas', 'diff_alta_trade'), ('aut', 'i_trade'), ('criterios', 'c4'), 
    ('casos_cc', 'es_susp_ext'), ('casos_cc', 'es_alta_prev_insp'), ('casos_cc', 'es_atipico'), ('casos_cc', 'es_trafico'),
    ('casos_cc', 'es_recaida_ip_denegada'), ('casos_cc', 'es_pl_empleado'), ('creadas', 'baja_mutua_7'),
    ('casos_cc', 'duracion'), ('casos_cc', 'dur_std'), ('casos_cc', 'v_std')
]

df = df.drop(columns=to_drop)

Eliminamos estas dos variables socioeconomicas porque practicamente tienen todos los valores `NaN`.

In [ ]:
to_drop = [
    ('socioeconomicas', 'gat_sector.38.21'), ('socioeconomicas', 'ght_sector.38.21')
]

df = df.drop(columns=to_drop)

In [ ]:
display(pd.DataFrame(df.isna().sum()).T)
print('NaN values:', pd.DataFrame(df.isna().sum()).T.sum(axis=1)[0])

In [ ]:
df = df.dropna()  

Extraemos del dataset la variable del resultado del detective. Crearemos a partir de esta dos nuevas variables que utilizaremos para la visualización y etiquetado que serán:
* Una binaria que identifique si es POS o no
* Una categorica de tres etiquetas que identifique, SinDetective/POS/NEG

In [ ]:
y_binary = [1 if i == 'POSA' or i == 'POSB' else 0 for i in df[('detectives','resultado')]]
y_cat = [1 if i == 'POSA' or i == 'POSB' else (0 if i == 'NEGA' else 2) for i in df[('detectives','resultado')]]

y_binary_desc = ['POS' if i == 'POSA' or i == 'POSB' else 'NS' for i in df[('detectives','resultado')]]
y_cat_desc = ['POS' if i == 'POSA' or i == 'POSB' else ('NEG' if i == 'NEGA' else 'NS') for i in df[('detectives','resultado')]]

df = df.drop(columns=[('detectives','resultado')])

---

Pasamos a dummies las variables categóricas

In [ ]:
make_dummies = list(set(df.columns) - set(float_cols))


for col in make_dummies:
    print(col)
    dummies = pd.get_dummies(df[col], prefix=col[1])
    multi_index = [['dummies'] * dummies.shape[1], list(dummies.columns)]
    dummies.columns = pd.MultiIndex.from_arrays(multi_index)
    df = pd.concat([df, dummies], axis=1)
    df = df.drop(columns=col)

In [ ]:
df.shape

----

Creamos un array booleano para seleccionar solo las muestras que han tenido detective (tanto NEG como POS)

In [ ]:
y_bool = [True if i == 1 or i == 0 else False for i in y_cat]

In [ ]:
labels = []
for i in y_cat:
    if i != 2:
        labels.append(i)

---

# Boruta

In [ ]:
X = df.iloc[y_bool, :].copy()
X = X.fillna(0)
X.columns = X.columns.droplevel(0)
X = X.reset_index(drop=True)

X.head(1)

In [ ]:
# rf = RandomForestClassifier(n_jobs=-1, class_weight='balanced', max_depth=5)
# feat_selector = BorutaPy(rf, n_estimators='auto', verbose=2, random_state=1, max_iter=300)
# feat_selector.fit(df.values, y_binary)

In [ ]:
# X_filtered = X.iloc[:, feat_selector.support_]
X_filtered = X.iloc[:,:]
X_filtered.shape

# Primer modelo

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_filtered, labels, shuffle=True, test_size = 0.3, random_state=1)

In [ ]:
# scaler = MinMaxScaler()
# scaler = scaler.fit(X_train)
# X_train = scaler.transform(X_train)
# X_test = scaler.transform(X_test)

In [ ]:
# pca = PCA(n_components=50)
# pca.fit(X_train)
# X_train = pca.transform(X_train)
# X_test = pca.transform(X_test)

# plt.plot(np.cumsum(pca.explained_variance_ratio_))

In [ ]:
params = HPOpt(X_train, X_test, y_train, y_test, False).fit(1000)

In [ ]:
clf = lgb.LGBMClassifier(**params)
clf.fit(X_train, y_train, 
 eval_set=[(X_train, y_train), (X_test, y_test)],
 callbacks = [
     lgb.early_stopping(10, verbose=0), 
     lgb.log_evaluation(period=0)
])

In [ ]:
lgb.plot_importance(clf, max_num_features=30, figsize=(10,8))

In [ ]:
probs = clf.predict_proba(X_test)[:, 1]
px.histogram(probs, nbins=1000, width=1000, height=400)

In [ ]:
fpr, tpr, thresholds = roc_curve(y_test,  probs, pos_label=1)
auc = roc_auc_score(y_test, probs)
gmeans = np.sqrt(tpr * (1-fpr))
ix = np.argmax(gmeans)

#create ROC curve
plt.plot(fpr,tpr,label="AUC="+str(auc))
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.legend(loc=4)
plt.show()

In [ ]:
THRESHOLD = thresholds[ix]
y_pred = np.where(probs >= THRESHOLD, 1, 0)
accuracy = accuracy_score(y_pred, y_test)
accuracy

In [ ]:
confusion_matrix = pd.crosstab(y_test, y_pred, rownames=['Actual'], colnames=['Predicted'])
sn.heatmap(confusion_matrix, annot=True, fmt='g')
plt.show()

In [ ]:
y_not = [False if i else True for i in y_bool]
sin_detective = df[y_not].copy()
sin_detective.columns = sin_detective.columns.droplevel(0)

In [ ]:
sin_probs = clf.predict_proba(sin_detective)
# px.histogram([i[1] for i in sin_probs], nbins=1000, width=1000, height=400)

In [ ]:
sin_pred = np.where(sin_probs[:,1] >= 0.6, 1, 0)
collections.Counter(sin_pred)

# Segundo modelo

In [ ]:
X_complete = pd.concat([sin_detective, X_filtered])
label = list(sin_pred) + labels

In [ ]:
to_drop = [
        ('diff_alta_adm_med'), ('diff_alta_adm_med_'), ('es_alta_adm_post_med_0'), ('es_alta_adm_post_med_1'), ('es_alta_adm_post_med2_0'), ('es_alta_adm_post_med2_1')
]

X_complete = X_complete.drop(columns=to_drop)

In [ ]:
X_train2, X_test2, y_train2, y_test2 = train_test_split(X_complete, label, shuffle=True, test_size = 0.2, random_state=0)

In [ ]:
params2 = HPOpt(X_train2, X_test2, y_train2, y_test2, False).fit(100)

In [ ]:
clf2 = lgb.LGBMClassifier(**params2)
clf2.fit(X_train2, y_train2, 
 eval_set=[(X_train2, y_train2), (X_test2, y_test2)],
 callbacks = [
     lgb.early_stopping(10, verbose=0), 
     lgb.log_evaluation(period=0)
])

In [ ]:
lgb.plot_importance(clf2, max_num_features=30, figsize=(10,8))

In [ ]:
probs2 = clf2.predict_proba(X_test2)
probs2 = [i[1] for i in probs2]
# px.histogram(probs2, nbins=1000, width=1000, height=400)

In [ ]:
fpr2, tpr2, thresholds2 = roc_curve(y_test2,  probs2, pos_label=1)
auc2 = roc_auc_score(y_test2, probs2)
gmeans2 = np.sqrt(tpr2 * (1-fpr2))
ix2 = np.argmax(gmeans2)

#create ROC curve
plt.plot(fpr2,tpr2,label="AUC="+str(auc2))
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.legend(loc=4)
plt.show()

In [ ]:
THRESHOLD2 = thresholds2[ix2]
y_pred2 = np.where(clf2.predict_proba(X_test2)[:,1] >= THRESHOLD2, 1, 0)
accuracy2 = accuracy_score(y_pred2, y_test2)
accuracy2

In [ ]:
confusion_matrix2 = pd.crosstab(y_test2, y_pred2, rownames=['Actual'], colnames=['Predicted'])
sn.heatmap(confusion_matrix2, annot=True, fmt='g')
plt.show()